In [38]:
import sys; sys.path.append('../../')
import numpy as np
from DataPreparation.DataPreparation import read_data
from utils import load_hyperparameters, save_model, get_metrics
from sklearn import svm
from sklearn.metrics import classification_report
from mlpath import mlquest as mlq
from ModelAnalysis import cross_validation
import pickle
model_name = 'SVM'

mlq.start_quest(model_name, table_dest="../../")

c:\Users\dell\AppData\Local\Programs\Python\Python38\lib\site-packages\mlpath\mlquest\mlquest.py:66: UserWarning: Attempting to start a run while another one is active may cause data overwrite
  if mlquest.active == True: warnings.warn("Attempting to start a run while another one is active may cause data overwrite")


# <font color="aqua">SVM</font> Model

### Read the data

In [39]:
x_data_d, y_data_d = mlq.l(read_data)(kind="Numerical")
opt_params = load_hyperparameters(model_name)

### Initiate model

In [46]:
#clf = mlq.l(svm.SVC)(**opt_params)
clf = mlq.l(svm.SVC)(kernel='linear', C=1186.866)

c:\Users\dell\AppData\Local\Programs\Python\Python38\lib\site-packages\mlpath\mlquest\mlquest.py:117: UserWarning: Attempting to log a function when no run is active will do nothing
  warnings.warn("Attempting to log a function when no run is active will do nothing")


### Evaluate Model Bias

In [47]:
clf.fit(x_data_d, y_data_d)
y_pred = clf.predict(x_data_d)
train_metrics = classification_report(y_data_d, y_pred, digits=3)
train_acc, train_wf1 = get_metrics(train_metrics)
print(train_metrics)

              precision    recall  f1-score   support

           0      0.987     0.993     0.990       152
           1      0.981     0.975     0.978       160
           2      0.991     0.994     0.992       324
           3      1.000     0.998     0.999       544

    accuracy                          0.993      1180
   macro avg      0.990     0.990     0.990      1180
weighted avg      0.993     0.993     0.993      1180



In [48]:
# perecentage of support vectors
sv = np.sum(clf.n_support_)/ len(x_data_d)
print("Percentage of support vectors: ",sv)
print("number of support vectors: ", np.sum(clf.n_support_))

Percentage of support vectors:  0.046610169491525424
number of support vectors:  55


### Evaluate Model Generalization

In [43]:
loo, kfold=cross_validation(clf, x_data_d, y_data_d, k=[10], n_repeats=[10], random_state=1)

loo_acc=loo.get('loo_acc')
loo_wf1=loo.get('loo_wf1')
repeated_10fold_acc, repeated_10fold_wf1 = kfold.get(f'10-Repeated 10-fold')[:2]    

# loo_report=loo.get('loo_report')
# repeated_10fold_report= kfold.get(f'10-Repeated 10-fold')[2]
# print(f'Leave-One-Out Report:\n {loo_report}')
# print(f'10-Repeated 10-fold Report:\n {repeated_10fold_report}')

loo_acc,10-Repeated 10-fold
0.9797,0.9831


loo_wf1,10-Repeated 10-fold
0.9797,0.9831


#### Save Model

In [44]:
save_model(model_name, clf)

### Tracking

In [45]:
mlq.log_metrics(train_acc, train_wf1,loo_acc, loo_wf1, repeated_10fold_acc, repeated_10fold_wf1,sv)
mlq.end_quest()
mlq.show_logs("SVM", table_dest="../../", last_k=6)